<a href="https://colab.research.google.com/github/An0816/PyTorchZeroToAll/blob/main/VanillaGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Overall Flow

1. 필요한 도구 선언  
2. train, test data load
3. model construction (Generator, Discriminator)
4. loss, optimizer 설정
5. train(forward, backward, update)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable 

import torchvision
from torchvision import transforms, datasets
from torchvision.utils import make_grid, save_image
from torch.utils.data import DataLoader

In [ ]:
train_data = datasets.MNIST(root = "MNIST/", train = True, download = True,
                            transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,),(0.3081,))                                                             
                            ]))
train_loader = DataLoader(dataset = train_data, batch_size = 64, shuffle = True, num_workers = 2, drop_last = True)

test_data = datasets.MNIST(root = "MNIST/", train = False, download = False, 
                           transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                           ]))

test_loader = DataLoader(dataset = test_data, batch_size = 64, shuffle = True, drop_last = True)


In [ ]:
class Generator(nn.Module):
    def __init__(self):

        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.model(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):

        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()    
        )
    
    def forward(self, x):
        x = self.model(x)
        return x


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
G = Generator().to(device)
D = Discriminator().to(device)

criterion = nn.BCELoss()
optim_G = optim.Adam(G.parameters(), lr = 1e-6)
optim_D = optim.Adam(D.parameters(), lr = 1e-6)

In [ ]:
for epoch in range(100):
    avg_loss = [0, 0]
    for x, target in train_loader:
        real = real.to(device) 
        real = torch.ones(x.shape[0], 1).to(device) 
        fake = torch.zeros(x.shape[0], 1).to(device)

        x = x.view(x.size(0), -1).to(device)

        noise = torch.randn(x.shape[0], 100, device = device)
        fake_img = G(noise) # MNIST 학습할 때의 y_pred느낌

        #training generator
        optim_G.zero_grad()
        loss_G = criterion(D(fake_img), real)
        loss_G.backward()
        optim_G.step()

        fake_img = fake_img.detach().to(device)

        #training discriminator
        optim_D.zero_grad()
        loss_D = criterion(D(torch.cat((x, fake_img))), torch.cat((real, fake)))
        loss_D.backward()
        optim_D.step()

        avg_loss[0] += loss_G
        avg_loss[1] += loss_D
    avg_loss[0] = avg_loss[0] / 64
    avg_loss[1] = avg_loss[1] / 64

    if epoch % 10 == 0:
        fake_img = fake_img.reshape([64, 1, 28, 28])
        img_grid = make_grid(fake_img, nrow = 10, normalize = True)
        save_image(img_grid, f"/content/drive/MyDrive/Deep Learning/result/%d.png"%(epoch))
        print(f"Epoch : {epoch}, Generator : {avg_loss[0]}, Discriminator : {avg_loss[1]}")